In [1]:
# Aloe: https://drive.google.com/file/d/1wxmiUdqMciuTOs0ouKEISl8-iTVXdOWn/view?usp=drive_link
!gdown --id 1wxmiUdqMciuTOs0ouKEISl8-iTVXdOWn
!unzip Aloe_images.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1wxmiUdqMciuTOs0ouKEISl8-iTVXdOWn
To: /content/Aloe_images.zip
100% 1.32M/1.32M [00:00<00:00, 13.4MB/s]
Archive:  Aloe_images.zip
  inflating: Aloe/Aloe_left_1.png    
  inflating: Aloe/Aloe_right_3.png   
  inflating: Aloe/Aloe_right_1.png   
  inflating: Aloe/Aloe_right_2.png   
  inflating: Aloe/Disparity_map_corr.png  


In [2]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [8]:
import cv2
import numpy as np

# 1. Semi-Global Block Matching (SGBM)
def sgbm_depth_estimation(left_img, right_img, min_disparity=0, num_disparities=64, block_size=7):
    # Đọc ảnh ở chế độ xám
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    # Kiểm tra xem ảnh đã được tải chính xác chưa
    if left is None or right is None:
        raise FileNotFoundError(f"One or both image files not found or could not be opened: {left_img}, {right_img}")

    # Khởi tạo bộ khớp SGBM
    sgbm = cv2.StereoSGBM_create(
        minDisparity=min_disparity,
        numDisparities=num_disparities,
        blockSize=block_size,
        P1=8 * 3 * block_size ** 2,
        P2=32 * 3 * block_size ** 2,
        disp12MaxDiff=1,
        uniquenessRatio=15,
        speckleWindowSize=100,
        speckleRange=32,
        preFilterCap=63,
        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
    )

    # Tính toán bản đồ chênh lệch
    disparity = sgbm.compute(left, right).astype(np.float32) / 16.0

    return disparity

# 2. Census Transform (helper function)
def census_transform(img, kernel_size=5):
    height, width = img.shape
    census_img = np.zeros((height, width), dtype=np.uint8)
    kernel_half = kernel_size // 2

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            center_pixel = img[y, x]
            census_value = 0
            bit = 1

            for v in range(-kernel_half, kernel_half + 1):
                for u in range(-kernel_half, kernel_half + 1):
                    if v == 0 and u == 0:
                        continue
                    census_value |= (img[y + v, x + u] < center_pixel) << bit
                    bit += 1

            census_img[y, x] = census_value

    return census_img

# 4. Sub-pixel Refinement (helper function)
def subpixel_refinement(disparity_map):
    refined_disparity = cv2.GaussianBlur(disparity_map, (5, 5), 0)
    return refined_disparity

# 5. Left-Right Consistency Check (helper function)
def left_right_consistency_check(left_disparity, right_disparity, threshold=1.5):  # Điều chỉnh ngưỡng
    height, width = left_disparity.shape
    consistency_map = np.zeros_like(left_disparity, dtype=np.uint8)

    for y in range(height):
        for x in range(width):
            d = left_disparity[y, x]
            if 0 <= x - d < width and abs(d - right_disparity[y, x - int(d)]) < threshold:
                consistency_map[y, x] = d
            else:
                consistency_map[y, x] = 0  # Mark inconsistent disparities

    return consistency_map

# 6. Post-processing (helper function)
def post_process_disparity(disparity_map):
    # Áp dụng lọc song phương để làm mịn bảo toàn cạnh
    disparity_bilateral = cv2.bilateralFilter(disparity_map, 9, 75, 75)

    return disparity_bilateral

# Main function to integrate all techniques
def depth_estimation_pipeline(left_img, right_img):
    # Step 1: SGBM to compute initial disparity
    disparity = sgbm_depth_estimation(left_img, right_img)

    # Step 2: Apply Census Transform (Optional)
    left_census = census_transform(cv2.imread(left_img, 0))
    right_census = census_transform(cv2.imread(right_img, 0))

    # Step 3: Sub-pixel refinement
    disparity_refined = subpixel_refinement(disparity)

    # Step 4: Compute right disparity map for consistency check
    disparity_right = sgbm_depth_estimation(right_img, left_img)

    # Step 5: Apply left-right consistency check
    consistency_map = left_right_consistency_check(disparity_refined, disparity_right)

    # Step 6: Post-process the disparity map
    final_disparity = post_process_disparity(consistency_map)

    # Normalize the disparity map for visualization
    final_disparity_normalized = cv2.normalize(final_disparity, None, 0, 255, cv2.NORM_MINMAX)
    final_disparity_normalized = np.uint8(final_disparity_normalized)

    # Save the final disparity map
    cv2.imwrite('final_disparity.png', final_disparity_normalized)
    cv2.imwrite('final_disparity_color.png', cv2.applyColorMap(final_disparity_normalized, cv2.COLORMAP_JET))

    print("Depth estimation pipeline completed.")
    return final_disparity_normalized

# Example usage
left_image_path = '/content/Aloe/Aloe_left_1.png'
right_image_path = '/content/Aloe/Aloe_right_1.png'
depth_map = depth_estimation_pipeline(left_image_path, right_image_path)

Depth estimation pipeline completed.


Depth estimation pipeline completed.
